In [11]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import yfinance as yf
import dash_bootstrap_components as dbc
import geopandas as gpd
import requests

In [2]:
# Load data
emissions = pd.read_csv('co2_emissions_kt_by_country.csv')
region = pd.read_csv('Methane_final.csv')
gdp = pd.read_csv('world_country_gdp_usd.csv')
pop = pd.read_csv('population_by_country_2020.csv')

region = region.drop(columns=['Unnamed: 0'])

gdp.columns = gdp.columns.str.replace(' ', '_')
gdp.columns = gdp.columns.str.lower()

pop = pop.rename(columns={'Country (or dependency)': 'country', 'Population (2020)': 'population', 'Density (P/Km²)': 'density', 
                          'Land Area (Km²)': 'land_area', 'Migrants (net)': 'migrants', 'Fert. Rate': 'fert_rate', 'Med. Age': 'med_age', 'Urban Pop %': 'urban_pop'})

In [27]:
geo = gpd.read_file('world_boundries.geojson')


In [30]:
geo = geo.drop(columns=['french_short', 'spanish_short', 'russian_short', 'chinese_short', 'arabic_short'])
geo.head()

,geo_point_2d,iso3_code,iso2_code,preferred_term,english_short,region_code,region_name,sub_region_code,sub_region_name,geometry
0,"{'lon': 117.30042819464269, 'lat': -2.23021409...",IDN,ID,Indonesia,Indonesia,142,Asia,35,South-eastern Asia,"MULTIPOLYGON (((123.21846 -10.80917, 123.19832..."
1,"{'lon': 96.5069210467498, 'lat': 21.1543190056...",MMR,MM,Myanmar,Myanmar,142,Asia,35,South-eastern Asia,"MULTIPOLYGON (((98.15582 9.87666, 98.13602 9.8..."
2,"{'lon': -60.968711171386566, 'lat': 13.8978660...",LCA,LC,Saint Lucia,St. Lucia,19,Americas,419,Latin America and the Caribbean,"MULTIPOLYGON (((-60.90028 13.77833, -60.93834 ..."
3,"{'lon': 28.046787741199427, 'lat': 53.53999776...",BLR,BY,Belarus,Belarus,150,Europe,151,Eastern Europe,"MULTIPOLYGON (((31.78388 52.10805, 31.73472 52..."
4,"{'lon': 11.797236422127188, 'lat': -0.59094448...",GAB,GA,Gabon,Gabon,2,Africa,202,Sub-Saharan Africa,"MULTIPOLYGON (((9.00778 -0.76778, 9.00222 -0.7..."


In [4]:
# merge emissions and gdp
merge = pd.merge(emissions, gdp, left_on='country_code', right_on='country_code', how='inner')
merge = merge.drop(columns=['country_name_y', 'year_y'])
merge = merge.rename(columns={'country_name_x': 'country_name', 'year_x': 'year'})
sec_merge = pd.merge(merge, region, left_on='country_name', right_on='country', how='inner')
final_merge = pd.merge(sec_merge, pop, left_on='country_name', right_on='country', how='inner')
emissions_gdp = final_merge.copy()


In [5]:
emissions_gdp = emissions_gdp.drop(columns=['country_x','country_y' , 'type', 'segment', 'reason', 'baseYear', 'notes'])
emissions_gdp = emissions_gdp.rename(columns={'country_name': 'country', 'value': 'co2_emissions', 'gdp_usd': 'gdp', 'gdp_per_capita_usd':'gdp_per_capita', 'emissions':'methane_emissions'})

emissions_gdp_2018 = emissions_gdp[emissions_gdp['year'] == 2018]

In [6]:
emissions_gdp

,country_code,country,year,co2_emissions,gdp,gdp_per_capita,region,methane_emissions,population,Yearly Change,Net Change,density,land_area,migrants,fert_rate,med_age,urban_pop,World Share
0,AGO,Angola,1960,550.050000,NaN,NaN,Africa,400.969574,33032075,3.27 %,1040977,26,1246700,6413.0,5.6,17,67 %,0.42 %
1,AGO,Angola,1960,550.050000,NaN,NaN,Africa,57.340000,33032075,3.27 %,1040977,26,1246700,6413.0,5.6,17,67 %,0.42 %
2,AGO,Angola,1960,550.050000,NaN,NaN,Africa,2.591680,33032075,3.27 %,1040977,26,1246700,6413.0,5.6,17,67 %,0.42 %
3,AGO,Angola,1960,550.050000,NaN,NaN,Africa,1.381880,33032075,3.27 %,1040977,26,1246700,6413.0,5.6,17,67 %,0.42 %
4,AGO,Angola,1960,550.050000,NaN,NaN,Africa,47.643501,33032075,3.27 %,1040977,26,1246700,6413.0,5.6,17,67 %,0.42 %
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249785,ZAF,South Africa,2019,439640.014648,4.199460e+11,6994.211654,Africa,10.874123,59436725,1.28 %,750420,49,1213090,145405.0,2.4,28,67 %,0.76 %
4249786,ZAF,South Africa,2019,439640.014648,4.199460e+11,6994.211654,Africa,1060.310059,59436725,1.28 %,750420,49,1213090,145405.0,2.4,28,67 %,0.76 %
4249787,ZAF,South Africa,2019,439640.014648,4.199460e+11,6994.211654,Africa,1239.029663,59436725,1.28 %,750420,49,1213090,145405.0,2.4,28,67 %,0.76 %
4249788,ZAF,South Africa,2019,439640.014648,4.199460e+11,6994.211654,Africa,42.407463,59436725,1.28 %,750420,49,1213090,145405.0,2.4,28,67 %,0.76 %


In [7]:
hist_data = emissions_gdp_2018[emissions_gdp_2018['co2_emissions'] > 200000]
px.histogram(hist_data, x='country', y='gdp_per_capita', color='region', title='GDP per capita per country')

In [8]:
px.histogram(hist_data, x='country', y='co2_emissions', color='region', title='CO2 emissions per country')

In [9]:
px.histogram(hist_data, x='country', y='methane_emissions', color='region', title='Methane emissions per country')